## Utitlity Functions
Useful functions to solve problems

In [1]:
import math
import time

def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        print( '%r  %2.2f ms' % (method.__name__, (te - ts) * 1000))
        return result
    return timed


def generate_primes_under(n=100):
    """
    Returns list of all prime numbers <= n
    """
    for possiblePrime in range(2, n + 1): # Assume number is prime until shown it is not. 
        isPrime = True
        for num in range(2, int(possiblePrime ** 0.5) + 1):
            if possiblePrime % num == 0:
                isPrime = False
                break        
        if isPrime:
            yield possiblePrime

            
def is_prime(n):
    if n == 2 or n == 3: return True
    if n < 2 or n%2 == 0: return False
    if n < 9: return True
    if n%3 == 0: return False
    r = int(n**0.5)
    f = 5
    while f <= r:
        if n%f == 0: return False
        if n%(f+2) == 0: return False
        f +=6
    return True

## Problem 60 - Prime pair sets

The primes 3, 7, 109, and 673, are quite remarkable. By taking any two primes and concatenating them in any order the result will always be prime. For example, taking 7 and 109, both 7109 and 1097 are prime. The sum of these four primes, 792, represents the lowest sum for a set of four primes with this property.

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [ ]:
from itertools import permutations, combinations


def is_remarkable_pair(p1_p2):
    if is_prime(int(''.join(map(str, p1_p2)))):
        if is_prime(int(''.join(map(str, p1_p2[::-1])))):
            return True
    return False

            
primes = tuple(generate_primes_under(10000))

@timeit
def solution(initial_no_of_primes=70):
    result = []
    
    p1_p2_r_pairs = [p for p in combinations(primes[:initial_no_of_primes], 2) if is_remarkable_pair(p)]
    for r_pair in p1_p2_r_pairs:
        K = []
        for prime in primes:
            if is_remarkable_pair((r_pair[0], prime)) and is_remarkable_pair((r_pair[1], prime)):
                K.append(prime)

        k1_k2_pairs = [k for k in combinations(K, 2) if is_remarkable_pair(k)]
        for k_pair in k1_k2_pairs:
            for k in K:
                if sorted((*r_pair, *k_pair, k)) in result:
                    continue
                if is_remarkable_pair((k_pair[0], k)) and is_remarkable_pair((k_pair[1], k)):
                    print(*r_pair, *k_pair, k)
                    result.append(sorted((*r_pair, *k_pair, k)))
                    # Remove return statement if you want all solutions
                    return result
        
    return result


# [13, 5197, 5701, 6733, 8389]
remarkable_primes = solution(1000)

def test_solution(remarkable_primes):
    for p in permutations(remarkable_primes, 2):
        number = int(str(p[0]) + str(p[1]))
        if not is_prime(number):
            print(number)
        assert is_prime(number)
        
(test_solution(s) for s in remarkable_primes)
min_sum = min([sum(s) for s in remarkable_primes])
min_r_set = [s for s in remarkable_primes if sum(s) == min_sum][0]
    
print(min_sum)
print(min_r_set)


## Problem 62 - Cubic permutations


The cube, `41063625 (345^3)`, can be permuted to produce two other cubes: `56623104 (384^3)` and `66430125 (405^3)`. In fact, `41063625` is the smallest cube which has exactly three permutations of its digits which are also cube.

Find the smallest cube for which exactly five permutations of its digits are cube.

In [ ]:
from itertools import permutations

In [ ]:
cubes = [x**3 for x in range(1, 200000)]

In [ ]:
@timeit
def solution(range_x, range_y):
    """
    127035954683
    """
    cubes = [x**3 for x in range(range_x, range_y)]
    for cube in cubes:
        tmp = []
        str_cube = str(cube)        
        for cube_2 in cubes[cubes.index(cube) + 1:]:
            if sorted(str(cube_2)) == sorted(str(cube)):
                tmp.append(cube_2)
        if len(tmp) == 4:
            print(cube, *tmp)
            return [cube, *tmp]

    
results = solution(2, 10000)

In [ ]:
def test(cube_permutations):
    for c in cube_permutations:
        assert c in cubes
    assert len({''.join(sorted(str(c))) for c in cube_permutations}) == 1
    print('successful', cube_permutations, min(cube_permutations))
            

test(results)

### Solution from project euler thread
Its much more efficient than mine...

In [ ]:

from time import process_time
start = process_time()

holder = {}
n = 1
while True:
    cube = tuple(sorted(list(str(n ** 3))))
    if cube in holder:
        holder[cube].append(n)
        if len(holder[cube]) == 5: # First one with 5 permutations.
            print(f'Answer is {holder[cube][0] ** 3}. Which took {process_time() - start} seconds.')
            break
    else:
        holder[cube] = [n]
    n += 1

## Problem 87 - Prime power triples

The smallest number expressible as the sum of a prime square, prime cube, and prime fourth power is 28. In fact, there are exactly four numbers below fifty that can be expressed in such a way:

```
28 = 2^2 + 2^3 + 2^4
33 = 3^2 + 2^3 + 2^4
49 = 5^2 + 2^3 + 2^4
47 = 2^2 + 3^3 + 2^4
```
How many numbers below fifty million can be expressed as the sum of a prime square, prime cube, and prime fourth power?

In [2]:
from itertools import permutations 

N = 50000000
p2_max = int(N**(1/2))
p3_max = int(N**(1/3))
p4_max = int(N**(1/4))

primes_2 = list(generate_primes_under(p2_max))
primes_3 = list(generate_primes_under(p3_max))
primes_4 = list(generate_primes_under(p4_max))

print('Number of primes it will loop through: {:,}'.format(len(primes_2)*len(primes_3)*len(primes_4)))

def solution(N):
    result = set()
    for p2 in primes_2:
        for p3 in primes_3:
            for p4 in primes_4:
                total = (p2**2) + (p3**3) + (p4**4)
                if total <= N:
                    result.add(total)
#                     print(p2, p3, p4, total)
    print(len(result))
    
solution(N=N)

Number of primes it will loop through: 1,524,532
1097343


## Problem 63 - Powerful digit counts

The 5-digit number, `16807=7^5`, is also a fifth power. Similarly, the 9-digit number, `134217728=8^9`, is a ninth power.

How many n-digit positive integers exist which are also an nth power?

In [33]:
i = 22
len(str(9**i))

21

In [28]:
"""
Number of digits in 10^i are always greater than i e.g. when i = 2, 10^2 = 100. Hence the maximum number is 9.

The maximum exponent is 21 since 9 ** 21 has 21 digits but 9 ** 22 has only 21 digits. 
"""
counter = 0
for x in range(1, 10):
    for i in range(1, 22):
        if len(str(x**i)) != i:
            break
        else:
            counter += 1
print(counter)

49
